Importing libraries

In [7]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import os
import mne
import shutil
from mne.preprocessing import annotate_amplitude

In [2]:
#a quick check of the raw data
raw_data_path = "/Users/folasewaabdulsalam/Signal_Processing/DS_Alzheimer_Project_Folasewa/preprocessed_dataset/sub-001/eeg/sub-001_task-eyesclosed_eeg.set"

raw_data = mne.io.read_raw_eeglab(raw_data_path, preload=True)

print (raw_data.info)


<Info | 8 non-empty values
 bads: []
 ch_names: Fp1, Fp2, F3, F4, C3, C4, P3, P4, O1, O2, F7, F8, T3, T4, T5, ...
 chs: 19 EEG
 custom_ref_applied: False
 dig: 22 items (3 Cardinal, 19 EEG)
 highpass: 0.0 Hz
 lowpass: 250.0 Hz
 meas_date: unspecified
 nchan: 19
 projs: []
 sfreq: 500.0 Hz
>


In [5]:
#taking a subset of the data 10 subjects for healthy, 10 for alzheimer

source_folder = "/Users/folasewaabdulsalam/Signal_Processing/DS_Alzheimer_Project_Folasewa/preprocessed_dataset"
target_folder = "/Users/folasewaabdulsalam/Signal_Processing/DS_Alzheimer_Project_Folasewa/filtered_subjects_short"


subject_ids = list(range(1, 11)) + list(range(37, 47))  # 10 subjects from healthy and 10 from alzheimer

# Loop through the selected subject IDs
for subject_id in subject_ids:
    subject_folder = f"sub-{str(subject_id).zfill(3)}"  # Format to sub-001, sub-002, etc.
    source_path = os.path.join(source_folder, subject_folder, "eeg", f"{subject_folder}_task-eyesclosed_eeg.set")

    
    if os.path.exists(source_path): # this checks if the source file exists
        shutil.copy(source_path, target_folder)  # this copies the file to the target folder
    else:
        print(f"File not found: {source_path}")


Signal Analysis

In [6]:
import glob
subject_path = "/Users/folasewaabdulsalam/Signal_Processing/DS_Alzheimer_Project_Folasewa/filtered_subjects_short"
set_files = glob.glob(f"{subject_path}/*.set")
filtered_output_preprocessed = "/Users/folasewaabdulsalam/Signal_Processing/DS_Alzheimer_Project_Folasewa/preprocessed_filtered_short"

In [18]:
def preprocess_file(set_path, output_path, n_components=19, noise_threshold=30e-6):
    """
    Preprocess an EEG file with optional noise detection, ICA, and ASR.

    Parameters:
    - set_path: Path to the input .set file.
    - output_path: Path to save the preprocessed file.
    - n_components: Number of components for ICA (default is 15).
    - noise_threshold: Noise threshold in volts (default is 30 µV, converted to volts).
    """
    print(f"Processing file: {set_path}")

    # Load the .set file
    raw_dataset = mne.io.read_raw_eeglab(set_path, preload=True)

    # Re-referencing using the common average referencing
    print("Applying average re-referencing...")
    raw_dataset.set_eeg_reference(ref_channels="average")

    # Apply band-pass filtering
    print("Applying band-pass filter 0.5Hz to 45Hz...")
    raw_dataset.filter(l_freq=0.5, h_freq=45)

    # Check for noise in the signal
    print("Checking for noise in the data...")
    # Compute standard deviation for each channel
    channel_stds = raw_dataset.get_data().std(axis=1)
    max_std = channel_stds.max()

    # If noise detected, apply ICA
    if max_std > noise_threshold:
        print(f"Noise detected! Maximum channel std: {max_std:.2e} V. Applying ICA...")

        # Perform ICA to remove artifacts
        ica = mne.preprocessing.ICA(n_components=n_components, random_state=42, max_iter="auto")
        ica.fit(raw_dataset)

        # Plot ICA components for manual inspection (optional)
        ica.plot_components()

        # Automatically exclude artifacts (if desired, or manually exclude specific components)
        ica.exclude = []  # You can update this based on manual inspection
        raw_dataset = ica.apply(raw_dataset)

    else:
        print(f"No significant noise detected (max std: {max_std:.2e} V). Skipping ICA.")

    # Apply ASR for automatic artifact rejection
    print("Applying Artifact Subspace Reconstruction (ASR)...")

    # Annotate bad segments based on the amplitude threshold
    annotations, _ = annotate_amplitude(
        raw_dataset,
        peak=17e-6,  # ASR threshold: 17 µV converted to volts
        min_duration=0.5,  # Minimum artifact duration (0.5 seconds)
    )

    # Apply annotations
    raw_dataset.set_annotations(annotations)

    # Exclude bad segments manually
    print("Excluding bad segments...")
    good_data = raw_dataset.get_data()
    for annotation in raw_dataset.annotations:
        onset_sample = int(annotation['onset'] * raw_dataset.info['sfreq'])
        duration_sample = int(annotation['duration'] * raw_dataset.info['sfreq'])
        good_data[:, onset_sample:onset_sample + duration_sample] = np.nan  # Mark bad segments as NaN

    # Interpolate or omit bad segments (optional)
    raw_dataset_clean = raw_dataset.copy()
    raw_dataset_clean._data = np.nan_to_num(good_data, nan=0.0)  # Replace NaNs with zeros (or interpolate)

    # Save the preprocessed file
    output_file = os.path.join(output_path, os.path.basename(set_path).replace(".set", "_preprocessed.fif"))
    print(f"Saving the preprocessed file to: {output_file}")
    raw_dataset_clean.save(output_file, overwrite=True)


In [19]:
for file in set_files:
    preprocess_file(file, filtered_output_preprocessed)


Processing file: /Users/folasewaabdulsalam/Signal_Processing/DS_Alzheimer_Project_Folasewa/filtered_subjects_short/sub-040_task-eyesclosed_eeg.set
Applying average re-referencing...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Applying band-pass filter 0.5Hz to 45Hz...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 3301 samples (6.602 s)



/var/folders/2r/22chk4fn06j2bd_2pbrv5vcm0000gn/T/ipykernel_46055/1850524937.py:14: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw_dataset = mne.io.read_raw_eeglab(set_path, preload=True)


Checking for noise in the data...
No significant noise detected (max std: 2.18e-05 V). Skipping ICA.
Applying Artifact Subspace Reconstruction (ASR)...
Finding segments below or above PTP threshold.


[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s


Excluding bad segments...
Saving the preprocessed file to: /Users/folasewaabdulsalam/Signal_Processing/DS_Alzheimer_Project_Folasewa/preprocessed_filtered_short/sub-040_task-eyesclosed_eeg_preprocessed.fif
Writing /Users/folasewaabdulsalam/Signal_Processing/DS_Alzheimer_Project_Folasewa/preprocessed_filtered_short/sub-040_task-eyesclosed_eeg_preprocessed.fif


...

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s
/var/folders/2r/22chk4fn06j2bd_2pbrv5vcm0000gn/T/ipykernel_46055/1850524937.py:76: RuntimeWarning: This filename (/Users/folasewaabdulsalam/Signal_Processing/DS_Alzheimer_Project_Folasewa/preprocessed_filtered_short/sub-037_task-eyesclosed_eeg_preprocessed.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw_dataset_clean.save(output_file, overwrite=True)


Closing /Users/folasewaabdulsalam/Signal_Processing/DS_Alzheimer_Project_Folasewa/preprocessed_filtered_short/sub-037_task-eyesclosed_eeg_preprocessed.fif
[done]
Processing file: /Users/folasewaabdulsalam/Signal_Processing/DS_Alzheimer_Project_Folasewa/filtered_subjects_short/sub-009_task-eyesclosed_eeg.set
Applying average re-referencing...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Applying band-pass filter 0.5Hz to 45Hz...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff 

/var/folders/2r/22chk4fn06j2bd_2pbrv5vcm0000gn/T/ipykernel_46055/1850524937.py:14: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw_dataset = mne.io.read_raw_eeglab(set_path, preload=True)
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s


Excluding bad segments...
Saving the preprocessed file to: /Users/folasewaabdulsalam/Signal_Processing/DS_Alzheimer_Project_Folasewa/preprocessed_filtered_short/sub-009_task-eyesclosed_eeg_preprocessed.fif
Writing /Users/folasewaabdulsalam/Signal_Processing/DS_Alzheimer_Project_Folasewa/preprocessed_filtered_short/sub-009_task-eyesclosed_eeg_preprocessed.fif
Closing /Users/folasewaabdulsalam/Signal_Processing/DS_Alzheimer_Project_Folasewa/preprocessed_filtered_short/sub-009_task-eyesclosed_eeg_preprocessed.fif
[done]


/var/folders/2r/22chk4fn06j2bd_2pbrv5vcm0000gn/T/ipykernel_46055/1850524937.py:76: RuntimeWarning: This filename (/Users/folasewaabdulsalam/Signal_Processing/DS_Alzheimer_Project_Folasewa/preprocessed_filtered_short/sub-009_task-eyesclosed_eeg_preprocessed.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw_dataset_clean.save(output_file, overwrite=True)
